In [1]:
import numpy as pd
import pandas as pd
import modelLoader
import dataLoader
import operator

In [2]:
pickle = modelLoader.fileManager()
l = dataLoader.Dataset()

In [3]:
#load model
model = pickle.loadIt('Models/version2.gz')
type(model)

sklearn.pipeline.Pipeline

In [4]:
#load no genre data 
noGenre = pickle.loadIt('Data/noGenreData.pkl')
noGenre.shape
#this data is a collection of all the none genre entries including 'unknown', '', ' ', [140], [144], '&'

(7235, 8)

In [5]:
noGenre.columns

Index(['Release Year', 'Title', 'Origin/Ethnicity', 'Director', 'Cast',
       'Genre', 'Wiki Page', 'Plot'],
      dtype='object')

In [6]:
#pd.options.display.max_rows = noGenre.shape[0]
noGenre.head()

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr..."
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov..."
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed..."
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...


In [7]:
noGenre.tail()

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
34252,2015,He's a Dragon,Russian,Indar Dzhendubaev,"Maria Poezzhaeva,\r\nMatvey Lykov,\r\nStanisla...",,https://en.wikipedia.org/wiki/He%27s_a_Dragon,"Princess Miroslava (Maria Poezzhaeva), called ..."
28419,2013,Rajesh Nair,Malayalam,"Rima Kallingal, Parthiban, Vijay Babu",Drama,[144],https://en.wikipedia.org/wiki/Escape_from_Uganda,"Shikha Samuel (Rima Kallingal), along with her..."
28417,2013,Kamal,Malayalam,"Jayaram, Remya Nambeesan",Drama,[140],https://en.wikipedia.org/wiki/Nadan,Nadan is the story of a popular drama troupe o...
27101,2017,R.S. Prasanna,Bollywood,"Ayushmann Khurrana, Bhumi Pednekar, Brijendra ...",Drama/Comedy,&,https://en.wikipedia.org/wiki/Shubh_Mangal_Sav...,The story is about Mudit Sharma (Ayushmann Khu...
27106,2017,Sanjay Chhel,Bollywood,"Rishi Kapoor, Paresh Rawal, Vir Das, Prem Chop...",Comedy/drama,&,https://en.wikipedia.org/wiki/Patel_Ki_Punjabi...,"The story is about a Punjabi, played by Rishi ..."


In [8]:
#make predicitons on unknown data
pred = model.predict(noGenre.Plot)
#create dataframe from predictions
dfPred = pd.DataFrame(pred, columns = ['predicted Genres'])
dfPred.shape[0]

7235

In [9]:
dfPred.head(100)

,predicted Genres
0,drama
1,drama
2,drama
3,drama
4,drama
...,...
95,drama
96,drama
97,drama
98,drama


In [10]:
#model seems biased towards drama, possibly because of more entries in original training set for drama 
#model also makes predictions for 'unknown' might be worth removing these for training? 

#count number of predictions for each genre
predGenreCount = {}

for genre in pred:
    if genre in predGenreCount:
        predGenreCount[genre] += 1
    else:
        predGenreCount[genre] = 1

In [11]:
print(predGenreCount)
#model only predicted drama, comedy and unknown from the data with no genres

{'drama': 3602, 'comedy': 109, 'unknown': 3524}


In [12]:
#load original dataset 
data = l.load_stored()

data.shape

(34886, 8)

In [13]:
genres = pd.unique(data.Genre)
genres.shape
#total number of unique genres

(182,)

In [14]:
#count how many times each genre appears in the original dataset
genCount = {}
for genre in data.Genre:
    if genre in genCount:
        genCount[genre] += 1
    else:
        genCount[genre] = 1
#order in descending order
genCount = dict( sorted(genCount.items(), key=operator.itemgetter(1),reverse=True))

genCount = pd.DataFrame([genCount])
genCount.head(genCount.shape[1])

,drama,comedy,unknown,action,horror,thriller,,romance,western,crime,...,kalyan,allari,rajendra,by,keiji,jidai,chanbara,mocatsu,splatter,dark
0,9398,6470,6083,1355,1209,1197,1149,1006,874,583,...,1,1,1,1,1,1,1,1,1,1


In [15]:
#transpose to make it easier to view
genCount.transpose()

,0
drama,9398
comedy,6470
unknown,6083
action,1355
horror,1209
...,...
jidai,1
chanbara,1
mocatsu,1
splatter,1


In [16]:
#drama comedy and unknwon appear far more than any other genre, i think this leads the model to being biased towards those genres
#might be worth removing the entries with no genre and retrain the model to see if this effects performance